In [21]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
import numpy as np
import torchvision.utils as vutils
import os

# 1. 환경 설정
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

batch_size = 64
print(f"batch_size: {batch_size}")

# 2. 데이터 준비 (Fashion MNIST)
transform = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

train_dataset = torchvision.datasets.FashionMNIST(
    root='./Fashion_MNIST_dataset',
    train=True,
    transform=transform,
    download=True
)
train_loader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=batch_size,
    shuffle=True
)


Using device: cuda
batch_size: 64


In [22]:
# 3. GAN 아키텍처 정의
class Generator(nn.Module):
    def __init__(self, noise_dim=100, img_dim=784):
        super(Generator, self).__init__()
        self.gen = nn.Sequential(
            nn.Linear(noise_dim, 128),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(128, 256),
            nn.BatchNorm1d(256, momentum=0.8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(256, 512),
            nn.BatchNorm1d(512, momentum=0.8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(512, 1024),
            nn.BatchNorm1d(1024, momentum=0.8),
            nn.LeakyReLU(0.2, inplace=True),

            nn.Linear(1024, img_dim),
            nn.Tanh()
        )
        
    def forward(self, x):
        return self.gen(x)

class Discriminator(nn.Module):
    def __init__(self, img_dim=784):
        super(Discriminator, self).__init__()
        self.disc = nn.Sequential(
            nn.Linear(img_dim, 512),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(512, 256),
            nn.LeakyReLU(0.2, inplace=True),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x):
        return self.disc(x)

In [23]:
# 4. 모델 초기화 및 설정
noise_dim = 100
img_dim = 28*28  # Fashion MNIST 이미지 크기
lr, b1, b2 = 3e-5, 0.5, 0.999
num_epochs = 100

generator = Generator(noise_dim, img_dim).to(device)
discriminator = Discriminator(img_dim).to(device)

criterion = nn.BCELoss()
optimizer_G = torch.optim.Adam(generator.parameters(), lr=lr, betas=(b1, b2))
optimizer_D = torch.optim.Adam(discriminator.parameters(), lr=lr, betas=(b1, b2))

# 5. 학습 루프 및 데이터 수집
# 5.1. 결과 저장을 위한 폴더 생성
results_dir = './results_GAN'
if not os.path.exists(results_dir):
    os.makedirs(results_dir)

G_losses = []
D_losses = []
sample_images = []

# fixed noise 생성
fixed_noise = torch.randn(16, noise_dim).to(device)

# 5.2. 학습 루프
for epoch in range(num_epochs):
    for batch_idx, (real, _) in enumerate(train_loader):
        real = real.view(-1, img_dim).to(device)
        batch_size = real.size(0)
        
        labels_real = torch.ones(batch_size, 1).to(device)
        labels_fake = torch.zeros(batch_size, 1).to(device)
        
        ## 판별자 학습 ##
        outputs = discriminator(real)
        loss_D_real = criterion(outputs, labels_real)
        
        noise = torch.randn(batch_size, noise_dim).to(device)
        fake = generator(noise)
        outputs = discriminator(fake.detach())
        loss_D_fake = criterion(outputs, labels_fake)
        
        loss_D = loss_D_real + loss_D_fake
        
        optimizer_D.zero_grad()
        loss_D.backward()
        optimizer_D.step()
        
        ## 생성자 학습 ##
        outputs = discriminator(fake)
        loss_G = criterion(outputs, labels_real)
        
        optimizer_G.zero_grad()
        loss_G.backward()
        optimizer_G.step()
        
    G_losses.append(loss_G.item())
    D_losses.append(loss_D.item())
    
    if epoch == 0 or (epoch+1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{num_epochs}]  Loss_D: {loss_D.item():.4f}  Loss_G: {loss_G.item():.4f}")

        with torch.no_grad():
            fake_images = generator(fixed_noise).reshape(-1, 1, 28, 28)
            sample_images.append(fake_images.cpu())


Epoch [1/100]  Loss_D: 1.2719  Loss_G: 0.6855
Epoch [10/100]  Loss_D: 1.2905  Loss_G: 0.7703
Epoch [20/100]  Loss_D: 1.3177  Loss_G: 0.7998
Epoch [30/100]  Loss_D: 1.3413  Loss_G: 0.7397
Epoch [40/100]  Loss_D: 1.3625  Loss_G: 0.7855
Epoch [50/100]  Loss_D: 1.3417  Loss_G: 0.7595
Epoch [60/100]  Loss_D: 1.2808  Loss_G: 0.8432
Epoch [70/100]  Loss_D: 1.3656  Loss_G: 0.7849
Epoch [80/100]  Loss_D: 1.3213  Loss_G: 0.7801
Epoch [90/100]  Loss_D: 1.3162  Loss_G: 0.8032
Epoch [100/100]  Loss_D: 1.2759  Loss_G: 0.8276


In [29]:
sample_images[0]

tensor([[[[-0.9512, -0.9866, -0.9761,  ..., -0.9615, -0.9957, -0.9552],
          [-0.9940, -0.9877, -0.9903,  ..., -0.9886, -0.8879, -0.9919],
          [-0.9705, -0.6791, -0.9752,  ..., -0.9862, -0.9255, -0.9742],
          ...,
          [-0.9956, -0.9939, -0.9933,  ..., -0.8381, -0.9655, -0.9902],
          [-0.9880, -0.9781, -0.9316,  ..., -0.8985, -0.9820, -0.9794],
          [-0.6999, -0.9402, -0.9916,  ..., -0.7995, -0.7922, -0.9876]]],


        [[[-0.8691, -0.9212, -0.9723,  ..., -0.9424, -0.8073, -0.9047],
          [-0.9525, -0.9058, -0.6687,  ..., -0.6922, -0.7546, -0.7721],
          [-0.9671, -0.8845, -0.9534,  ..., -0.9641, -0.8783, -0.9311],
          ...,
          [-0.9336, -0.8313, -0.8466,  ..., -0.8205, -0.2713, -0.8655],
          [-0.9463, -0.8165, -0.2278,  ..., -0.8046, -0.9423, -0.8758],
          [-0.8292, -0.9343, -0.8865,  ..., -0.9530, -0.9619, -0.7946]]],


        [[[-0.9179, -0.9683, -0.9876,  ..., -0.9470, -0.8778, -0.9942],
          [-0.9613, -0.962

In [25]:
# 6. 시각화
# 6.1. 손실 곡선 시각화 및 저장
plt.figure(figsize=(10,5))
plt.title("Generator and Discriminator Loss During Training")
plt.plot(G_losses, label="Generator Loss")
plt.plot(D_losses, label="Discriminator Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.legend()
plt.savefig(os.path.join(results_dir, "loss_curve.png"))
plt.close()

# 6.2. 생성된 이미지 시각화 및 저장
for idx, images in enumerate(sample_images):
    grid = vutils.make_grid(images, nrow=4, normalize=True)
    plt.figure(figsize=(8,8))
    plt.title(f"Generated Images at Epoch {max(idx*10, 1)}")
    plt.imshow(np.transpose(grid, (1,2,0)))
    plt.axis("off")
    # 이미지 저장
    plt.savefig(os.path.join(results_dir, f"generated_epoch_{max(idx*10, 1)}.png"))
    plt.close()